In [364]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [365]:
def dispatch(ctime, location, driver, name, p1=None, p2=None):
        #d = retrieveDriver(driver, es)
        
        p = scanPassenger(location, es)
        
        dDoc = {"ctime": ctime, "location": location, "status": "idle"}
        if p:
            doc = {"status": "pickup", "driver": driver, "ctime": ctime}
            updatePassenger(p['id'], doc, es)

            if p1: 
                dDoc['p2'] = p['id']
            else:
                dDoc['p1'] = p['id']
                
            dDoc['status'] = "pickup"
            dDoc['destination'] = p['location']
            dDoc['destinationid'] = p['id']
            

        updateDriver(driver, dDoc, es)
        
        bulk = (1, '{{doc: {}}}'.format(json.dumps(dDoc)))
        return (bulk)

In [423]:
window = 'now-2h'

q = {'size': 0,
    'query': { 'terms': {'status': ['arrived']} },
    'filter': {'range': { 'ctime': { 'gt': window }} 
        },
    }
res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])



In [424]:
res['hits']["total"]

1351

In [405]:
dDoc

{'ctime': 'what', 'iam': 'hungry', 'location': 'ever', 'status': 'is'}